In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
from PIL import ImageOps
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
#import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Dropout, Conv2D, MaxPooling2D, Flatten, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import optimizers
%matplotlib inline

In [2]:
def euclidean_distance(y_true, y_pred):
    return K.sqrt(K.maximum(K.sum(K.square(y_pred - y_true), axis=-1, keepdims=True), K.epsilon()))

In [3]:
# feature extraction from input image
img = Input(shape = (150,150,3), name="input_image")

x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv1")(img)
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv2")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1')(x)
    
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv3")(x)
x = Conv2D(64, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv4")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2')(x)
   
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv5")(x)
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv6")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool3')(x)
    
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv7")(x)
x = Conv2D(128, (3, 3), padding="same", strides=(1, 1), activation="relu", name="conv8")(x)
x = BatchNormalization()(x)
    
x = Flatten()(x)
x = Dropout(0.75, noise_shape=None, seed=None)(x)
x = Dense(1024, name='FC1')(x)
out = Dense(13, name='loss')(x)

# create model
model = Model(inputs=[img], outputs=[out])

# set output types
target = tf.compat.v1.placeholder(dtype='float32', shape=(13,1)) 

# get model summary
#model.summary()

# compile model
model.compile(optimizer=optimizers.legacy.Adam(lr=0.0015, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss=euclidean_distance)
#plot_model(model, to_file='model.png')


/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
import numpy as np
data_path = 'your pth/'
Fx = np.load(data_path+"fx.npy", mmap_mode='r')
Fy = np.load(data_path+"fy.npy", mmap_mode='r')
U0 = np.load(data_path+"U0.npy", mmap_mode='r')
V0 = np.load(data_path+"V0.npy", mmap_mode='r')
Disparity = np.load(data_path+"disparity.npy", mmap_mode='r')
Tx = np.load(data_path+"x.npy", mmap_mode='r')
Ty = np.load(data_path+"y.npy", mmap_mode='r') 
Tz = np.load(data_path+"z.npy", mmap_mode='r')
Baseline = np.load(data_path+"baseline.npy", mmap_mode='r')
Pitch = np.load(data_path+"pitch.npy", mmap_mode='r')
X = np.load(data_path+"xworld.npy", mmap_mode='r')
Y = np.load(data_path+"yworld.npy", mmap_mode='r')
Z = np.load(data_path+"zworld.npy", mmap_mode='r')
Left_images = np.load(data_path+"left_img.npy", mmap_mode='r')
Right_images = np.load(data_path+"right_img.npy", mmap_mode='r')

In [5]:
X = []
Y = []

for idx in range(len(Left_images)):
                
    l_im = Left_images[idx]
    #l_im = l_im[:,:,0]
    #r_im = Right_images[idx]
    #r_im = r_im[:,:,0]
    #training_image = np.dstack((l_im, r_im))
        
    X.append(l_im)
                
    Y.append([Fx[idx], Fy[idx], U0[idx], V0[idx], Baseline[idx], Disparity[idx], Tx[idx], Ty[idx], Tz[idx], Pitch[idx], Xw[idx], Yw[idx], Zw[idx]])
  

In [6]:
print ("dataset: ",np.shape(X))

dataset:  (63600, 150, 150, 3)


In [7]:
import os
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
#from utils_regressor_focal_dist import RotNetDataGenerator, angle_error, CustomModelCheckpoint
from keras import optimizers
import numpy as np
import glob, math
from shutil import copyfile
import datetime, random
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

model_name = 'model_multi_class/'
SAVE = "new_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '/'
# Save
output_folder = SAVE + model_name
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

output_log = output_folder + "Log/"
if not os.path.exists(output_log):
    os.makedirs(output_log)

output_weight = output_folder + "Best/"
if not os.path.exists(output_weight):
    os.makedirs(output_weight)


input_shape = (150, 150, 3)

tensorboard = TensorBoard(log_dir=output_log)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)

csv_logger = CSVLogger('training.log')

checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath=output_weight + "weights_{epoch:02d}_{val_loss:.2f}.h5",
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True
)

history = model.fit(
    x=[X[:int(len(X)*0.7)]],
    y=[Y[:int(len(Y)*0.7)]],
    batch_size=32,
    epochs=100,
    verbose=2,
    shuffle=True,
    initial_epoch=0,
    validation_data=([X[int(len(X)*0.7):]], [Y[int(len(Y)*0.7):]]),
    callbacks=[tensorboard, checkpointer, csv_logger]
)

Train on 44520 samples, validate on 19080 samples
Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


44520/44520 - 104s - loss: 77.2198 - val_loss: 107.7818 - 104s/epoch - 2ms/sample
Epoch 2/100
44520/44520 - 95s - loss: 66.4228 - val_loss: 136.0052 - 95s/epoch - 2ms/sample
Epoch 3/100
44520/44520 - 93s - loss: 64.0067 - val_loss: 70.1479 - 93s/epoch - 2ms/sample
Epoch 4/100
44520/44520 - 92s - loss: 62.5799 - val_loss: 82.6838 - 92s/epoch - 2ms/sample
Epoch 5/100
44520/44520 - 92s - loss: 62.0992 - val_loss: 90.3683 - 92s/epoch - 2ms/sample
Epoch 6/100
44520/44520 - 93s - loss: 61.5849 - val_loss: 68.7803 - 93s/epoch - 2ms/sample
Epoch 7/100
44520/44520 - 92s - loss: 61.1053 - val_loss: 83.0810 - 92s/epoch - 2ms/sample
Epoch 8/100
44520/44520 - 94s - loss: 60.8091 - val_loss: 98.5848 - 94s/epoch - 2ms/sample
Epoch 9/100
44520/44520 - 92s - loss: 60.5217 - val_loss: 83.2119 - 92s/epoch - 2ms/sample
Epoch 10/100
44520/44520 - 93s - loss: 60.3293 - val_loss: 89.5054 - 93s/epoch - 2ms/sample
Epoch 11/100
44520/44520 - 93s - loss: 60.5086 - val_loss: 184.1392 - 93s/epoch - 2ms/sample
Epoc